In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter, OrderedDict
from itertools import combinations

In [2]:
data = pd.read_csv('movie_bd_v5.csv', encoding = 'utf-8')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
43,tt2224026,135000000,368871007,Home,Jim Parsons|Rihanna|Steve Martin|Jennifer Lope...,Tim Johnson,Worlds Collide,When Earth is taken over by the overly-confide...,94,Fantasy|Comedy|Animation|Science Fiction|Family,Twentieth Century Fox Film Corporation|DreamWo...,3/18/2015,6.9,2015
1398,tt0398165,82000000,190320568,The Longest Yard,Adam Sandler|Chris Rock|Burt Reynolds|Michael ...,Peter Segal,"If you can't get out, get even.","Pro quarter-back, Paul Crewe and former colleg...",113,Comedy|Drama,Paramount Pictures|Columbia Pictures Corporati...,5/19/2005,6.2,2005
1875,tt0219854,65000000,69700000,The Kid,Bruce Willis|Spencer Breslin|Emily Mortimer|Li...,Jon Turteltaub,Nobody ever grows up quite like they imagined.,Powerful businessman Russ Duritz is self-absor...,104,Fantasy|Comedy|Family,Walt Disney Pictures,7/7/2000,5.9,2000
1320,tt0399201,126000000,162949164,The Island,Ewan McGregor|Scarlett Johansson|Djimon Hounso...,Michael Bay,Your time will come...,Lincoln Six-Echo is a resident of a seemingly ...,136,Action|Thriller|Science Fiction|Adventure,DreamWorks SKG|Warner Bros.|Parkes/MacDonald P...,7/20/2005,6.4,2005
894,tt0233469,85000000,78382433,Collateral Damage,Arnold Schwarzenegger|Francesca Neri|Elias Kot...,Andrew Davis,What would you do if you lost everything?,Firefighter Gordon Brewer is plunged into the ...,108,Action|Thriller|Drama,David Foster Productions|Bel Air Entertainment...,2/6/2002,5.5,2002


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [5]:
answers = {} # создадим словарь для ответов

data['release_date'] = pd.to_datetime(data['release_date'], format='%m/%d/%Y') #Change time in datetime format
profit_values = data.apply(lambda x: x.revenue - x.budget, axis = 1) #Create column "profit" = revenue - budget
data.insert(3, 'profit', profit_values)
data["title_length"]= data["original_title"].str.len() #Create column length of original title
data['totalwords'] = data['overview'].str.count(' ') + 1 #Create column counts words in the overview
month = data.release_date.dt.month #create column release_month
data.insert(12, 'release_month', month)

In [6]:
#Functions:
def counter(df_column):
    '''This function counts the entries in the column'''
    value_max = Counter()
    for x in df_column:
        value_max += Counter(x)
    return value_max.most_common(10)

In [7]:
data.head(2)

,imdb_id,budget,revenue,profit,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_month,release_date,vote_average,release_year,title_length,totalwords
0,tt0369610,150000000,1513528810,1363528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6,2015-06-09,6.5,2015,14,26
1,tt1392190,150000000,378436354,228436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5,2015-05-13,7.1,2015,18,110


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [8]:
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides'

In [9]:
# код для решения данного вопроса:
data[data.budget == data.budget.max()].original_title

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

# 2. Какой из фильмов самый длительный (в минутах)?

In [10]:
answers['2'] = 'Gods and Generals'

In [11]:
data[data.runtime == data.runtime.max()].original_title

1157    Gods and Generals
Name: original_title, dtype: object

# 3. Какой из фильмов самый короткий (в минутах)?





In [12]:
answers['3'] = 'Winnie the Pooh'

In [13]:
data[data.runtime == data.runtime.min()].original_title

768    Winnie the Pooh
Name: original_title, dtype: object

# 4. Какова средняя длительность фильмов?


In [14]:
answers['4'] = '109.6585494970884'

In [15]:
data.runtime.mean()

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [16]:
answers['5'] = '107.0'

In [17]:
data.runtime.median()

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [18]:
answers['6'] = 'Avatar'

In [19]:
data[data.profit == data.profit.max()].original_title

239    Avatar
Name: original_title, dtype: object

# 7. Какой фильм самый убыточный? 

In [20]:
answers['7'] = 'The Lone Ranger'

In [21]:
data[data.profit == data.profit.min()].original_title

1245    The Lone Ranger
Name: original_title, dtype: object

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [22]:
answers['8'] = '1478'

In [23]:
data[data.revenue > data.budget].original_title.count()

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [24]:
answers['9'] = 'The Dark Knight'

In [25]:
data[data.release_year == 2008][['original_title', 'profit', 'release_year']].sort_values(['profit'], ascending=False).head(1)

,original_title,profit,release_year
599,The Dark Knight,816921825,2008


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [26]:
answers['10'] = 'The Lone Ranger'

In [27]:
data[(2012 <= data.release_year) & (data.release_year <= 2014)][['original_title', 'profit', 'release_year']].sort_values(['profit'], ascending=True).head(1)

,original_title,profit,release_year
1245,The Lone Ranger,-165710090,2013


# 11. Какого жанра фильмов больше всего?

In [28]:
answers['11'] = 'Drama, 782'

In [29]:
genres_df = data.copy()
genres_df.genres = genres_df.genres.str.split('|')
print(counter(genres_df.genres))

[('Drama', 782), ('Comedy', 683), ('Thriller', 596), ('Action', 582), ('Adventure', 415), ('Crime', 315), ('Romance', 308), ('Family', 260), ('Science Fiction', 248), ('Fantasy', 222)]


ВАРИАНТ 2

In [30]:
#this is less effective function
genres_dict = {}
for row in genres_df.genres:
    for genre in row:
        if genre in genres_dict:
            genres_dict[genre] += 1
        else:
            genres_dict[genre] = 1
print(sorted(genres_dict.items(), key=lambda x: x[1], reverse=True))

[('Drama', 782), ('Comedy', 683), ('Thriller', 596), ('Action', 582), ('Adventure', 415), ('Crime', 315), ('Romance', 308), ('Family', 260), ('Science Fiction', 248), ('Fantasy', 222), ('Horror', 176), ('Mystery', 168), ('Animation', 139), ('Music', 64), ('History', 62), ('War', 58), ('Western', 19), ('Documentary', 8), ('Foreign', 2)]


# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [31]:
answers['12'] = 'Adventure'

In [32]:
new_genre = genres_df.explode('genres')
new_genre.groupby(['genres'])['profit'].sum().sort_values(ascending=False).head(1)

genres
Adventure    86070466981
Name: profit, dtype: int64

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [33]:
answers['13'] = 'Peter Jackson'

In [34]:
director_df = data.copy()
director_df.director = director_df.director.str.split('|')
new_director = director_df.explode('director')
new_director.groupby(['director'])['revenue'].sum().sort_values(ascending=False).head(1)

director
Peter Jackson    6490593685
Name: revenue, dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [35]:
answers['14'] = 'Robert Rodriguez'

In [36]:
genres_df = director_df.explode('director')
genres_director = genres_df[genres_df.genres.str.contains('Action', na = False)]
genres_director['director'].value_counts().head(1)

Robert Rodriguez    9
Name: director, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году?¶

In [37]:
answers['15'] = 'Chris Hemsworth'

In [38]:
cast_df = data.copy()
cast_df.cast = cast_df.cast.str.split('|')
cast_df = cast_df.explode('cast')
cast2012 = cast_df[cast_df.release_year == 2012]
cast2012.groupby('cast')['revenue'].sum().sort_values(ascending = False).head(1)

cast
Chris Hemsworth    2027450773
Name: revenue, dtype: int64

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [39]:
answers['16'] = 'Matt Damon'

In [40]:
cast_budget = cast_df[cast_df.budget > cast_df.budget.median()]
cast_budget.groupby('cast')['original_title'].count().sort_values(ascending = False).head(1)

cast
Matt Damon    22
Name: original_title, dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [41]:
answers['17'] = 'Drama'

In [42]:
genres_df = data.copy()
genres_df.genres = genres_df.genres.str.split('|')
genres_df = genres_df.explode('genres')
profit_genre = genres_df[genres_df.profit > 0]
profit_genre.groupby(['genres']).imdb_id.count().sort_values(ascending = False).head(1)

genres
Drama    560
Name: imdb_id, dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [43]:
answers['18'] = 'K-19: The Widowmaker'

In [44]:
data_paramount = data[data.production_companies.str.contains('Paramount Pictures', na=False)]
data_paramount[data_paramount.profit == data_paramount.profit.min()].original_title

925    K-19: The Widowmaker
Name: original_title, dtype: object

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [45]:
answers['19'] = '2015'

In [46]:
data.groupby('release_year')['profit'].sum().sort_values(ascending = False).head(1)

release_year
2015    18668572378
Name: profit, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [47]:
answers['20'] = '2014'

In [48]:
data_warner = data[data.production_companies.str.contains('Warner Bros', na=False)]
data_warner.groupby('release_year')['profit'].sum().sort_values(ascending = False).head(1)

release_year
2014    2295464519
Name: profit, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [49]:
answers['21'] = 'September'

In [50]:
grouped_month = data.groupby('release_month')
grouped_month.release_month.count().sort_values(ascending = False).head(1)

release_month
9    227
Name: release_month, dtype: int64

In [51]:
grouped_month = data.groupby('release_month').imdb_id.count()
grouped_month.sort_values(ascending = False).head(1)

release_month
9    227
Name: imdb_id, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [52]:
answers['22'] = '450'

In [53]:
grouped_month[6] + grouped_month[7] + grouped_month[8]

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [54]:
answers['23'] = 'Peter Jackson'

In [55]:
data[(data.release_month == 12) | (data.release_month == 1) | (data.release_month == 2)].director.value_counts().head(1)

Peter Jackson    7
Name: director, dtype: int64

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [56]:
answers['24'] = 'Four By Two Productions'

In [57]:
data["title_length"]= data["original_title"].str.len()
title_length = data.copy()
title_length.production_companies = title_length.production_companies.str.split('|')
new_length = title_length.explode('production_companies')
new_length.groupby(['production_companies'])['title_length'].mean().sort_values(ascending = False).head(1)

production_companies
Four By Two Productions    83.0
Name: title_length, dtype: float64

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [58]:
answers['25'] = 'Midnight Picture Show'

In [59]:
new_length.groupby(['production_companies'])['totalwords'].mean().sort_values(ascending = False).head(1)

production_companies
Midnight Picture Show    175.0
Name: totalwords, dtype: float64

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [60]:
answers['26'] = '1'

In [61]:
new_data = data[['original_title', 'vote_average']]
best_films = new_data.sort_values(by = 'vote_average', ascending = False).head(18)

In [62]:
ans1 = ['Inside Out', 'The Dark Knight', '12 Years a Slave']
ans2 = ['BloodRayne', 'The Adventures of Rocky & Bullwinkle']
ans3 = ['Batman Begins', 'The Lord of the Rings: The Return of the King', 'Upside Down']
ans4 = ['300', 'Lucky Number Slevin', 'Kill Bill: Vol. 1']
ans5 = ['Upside Down', 'Inside Out', 'Iron Man']
answer = {}
for i in range(1, 6):
    answer[i] = 0
for title in best_films.original_title:
    if title in ans1:
        answer[1] += 1
    elif title in ans2:
        answer[2] +=1
    elif title in ans3:
        answer[3] +=1
    elif title in ans4:
        answer[4] +=1
    elif title in ans5:
        answer[5] +=1
print('The correct answer is ', max(answer, key=answer.get))

The correct answer is  1


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [63]:
answers['27'] = 'Daniel Radcliffe, Rupert Grint'

In [64]:
actors = data.copy()
actors['actor'] = actors.cast.str.split('|')
liste = []
for cast in actors['actor']:
    liste.append(list(combinations(cast,2)))
counter = {}
for row in liste:
    for pair in row:
        if pair in counter.keys():
            counter[pair] += 1
        else:
            counter[pair] = 1
print(max(counter, key=counter.get))

('Daniel Radcliffe', 'Rupert Grint')


# Submission

In [65]:
# в конце можно посмотреть свои ответы к каждому вопрос
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides',
 '2': 'Gods and Generals',
 '3': 'Winnie the Pooh',
 '4': '109.6585494970884',
 '5': '107.0',
 '6': 'Avatar',
 '7': 'The Lone Ranger',
 '8': '1478',
 '9': 'The Dark Knight',
 '10': 'The Lone Ranger',
 '11': 'Drama, 782',
 '12': 'Adventure',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Drama',
 '18': 'K-19: The Widowmaker',
 '19': '2015',
 '20': '2014',
 '21': 'September',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': '1',
 '27': 'Daniel Radcliffe, Rupert Grint'}

In [66]:
# и убедиться что ни чего не пропустил)
len(answers)

27